In [1]:
import numpy as np
import tensorflow as tf

/home/aditya/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Getting Tensorflow Data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets('MNIST_data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### Function for MaxPooling(size=2x2, stride=2x2)

In [3]:
def max_pool(X):
    return tf.nn.max_pool(X,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

### Function for Convolution and adding biases(strides=1,filter= 5x5)

In [4]:
def convolution_layer(X,w,b):
    z= tf.nn.conv2d(X,w,strides=[1,1,1,1],padding='SAME')
    return tf.nn.relu(z+b)

### Function for fully connected layer

In [5]:
def fully_connected_layer(X,w,b):
    z=tf.matmul(X,w)+b
    return tf.nn.relu(z)

### Initializing weights and biases for Convolution and Maxpooling layer

In [6]:
w1=tf.Variable(tf.truncated_normal([3,3,1,16]))
b1=tf.Variable(tf.truncated_normal([16]))
w2=tf.Variable(tf.truncated_normal([3,3,16,32]))
b2=tf.Variable(tf.truncated_normal([32]))
batch_size=100

### Creating placeholder for Input and Target and the reshaping input for convolution.

In [7]:
X=tf.placeholder(tf.float32,shape=[None,784])
y_true=tf.placeholder(tf.float32,shape=[None,10])
X_image=tf.reshape(X,[-1,28,28,1])

### Appylying two layers of subsequent convolution. Initializing weights and biases for fully connected layer. Applying Fully Connected Layer. Predicting the output 

In [8]:
conv1=convolution_layer(X_image,w1,b1)
max_pool1=max_pool(conv1)
conv2=convolution_layer(max_pool1,w2,b2)
max_pool2=max_pool(conv2)
a,b,c,d=max_pool2.shape
max_pool2=tf.reshape(max_pool2,[-1,b*c*d])
w3=tf.Variable(tf.truncated_normal([1568,1024]))
b3=tf.Variable(tf.truncated_normal([1024]))
fcn1=fully_connected_layer(max_pool2,w3,b3)
w4=tf.Variable(tf.truncated_normal([1024,10]))
b4=tf.Variable(tf.truncated_normal([10]))
y_pred=(tf.matmul(fcn1,w4)+b4)

### Applying Softmax Regression and calculating loss with cross entropy and train the model with Adam Optimizer

In [9]:
with tf.name_scope('Loss'):
    loss_op=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))
with tf.name_scope('Adam'):
    train_op=tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss_op)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
with tf.name_scope('Accuracy'):
    prediction=tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
    accuracy=tf.reduce_mean(tf.cast(prediction,tf.float32))

In [11]:
tf.summary.scalar("loss", loss_op)
tf.summary.scalar("accuracy", accuracy)
merged_summary_op = tf.summary.merge_all()

In [12]:
saver=tf.train.Saver()

### Running the model. Batch_size=100. We train over 1000 batches. on the last batch we have an accuracy of 96% and 93.4% accuracy on the 1st 500 training images.

In [13]:
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter('./output1', sess.graph)
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        batch_x,batch_y=mnist.train.next_batch(100)
        sess.run([train_op,loss_op],feed_dict={X:batch_x,y_true:batch_y})  
        print(sess.run([accuracy],feed_dict={X:batch_x,y_true:batch_y}))
        _, c, summary = sess.run([train_op, loss_op, merged_summary_op],feed_dict={X: batch_x, y_true: batch_y})
        summary_writer.add_summary(summary,i)
    print(sess.run(accuracy,feed_dict={X: mnist.test.images[:500,:], y_true: mnist.test.labels[:500,:]}))
    saver.save(sess,'cnn1/checkpoint1.ckpt')
  

[0.25]
[0.2]
[0.23]
[0.45]
[0.45]
[0.53]
[0.58]
[0.61]
[0.49]
[0.61]
[0.81]
[0.76]
[0.71]
[0.64]
[0.71]
[0.78]
[0.82]
[0.82]
[0.73]
[0.75]
[0.86]
[0.83]
[0.78]
[0.74]
[0.82]
[0.84]
[0.82]
[0.86]
[0.89]
[0.8]
[0.82]
[0.84]
[0.85]
[0.83]
[0.87]
[0.86]
[0.86]
[0.91]
[0.9]
[0.88]
[0.88]
[0.89]
[0.84]
[0.9]
[0.9]
[0.84]
[0.85]
[0.85]
[0.92]
[0.89]
[0.81]
[0.89]
[0.88]
[0.91]
[0.9]
[0.9]
[0.86]
[0.9]
[0.85]
[0.9]
[0.88]
[0.84]
[0.91]
[0.85]
[0.89]
[0.89]
[0.84]
[0.89]
[0.85]
[0.95]
[0.86]
[0.91]
[0.88]
[0.92]
[0.91]
[0.93]
[0.84]
[0.88]
[0.88]
[0.9]
[0.9]
[0.94]
[0.89]
[0.93]
[0.91]
[0.9]
[0.93]
[0.91]
[0.87]
[0.91]
[0.91]
[0.92]
[0.9]
[0.91]
[0.89]
[0.94]
[0.87]
[0.92]
[0.94]
[0.92]
[0.86]
[0.9]
[0.92]
[0.86]
[0.91]
[0.91]
[0.88]
[0.95]
[0.92]
[0.95]
[0.85]
[0.93]
[0.94]
[0.92]
[0.93]
[0.88]
[0.93]
[0.93]
[0.91]
[0.85]
[0.9]
[0.93]
[0.9]
[0.94]
[0.94]
[0.9]
[0.94]
[0.96]
[0.89]
[0.92]
[0.97]
[0.94]
[0.89]
[0.89]
[0.94]
[0.93]
[0.96]
[0.94]
[0.91]
[0.93]
[0.93]
[0.93]
[0.94]
[0.92]
[0.91]
[0